In [ ]:
def init_collaborative_filtering_user_based (ratings_matrix):
    #Recibimos la matriz "ratings_matrix" que está compuesta por el cruce entre todos los síntomas y todos las
    #enfermedades; y que tiene los valores 0 (cuando no hay relación) y 1,2,3 y 4 en función de la frecuencia
    #de releación entre el síntoma y la enfermedad. Por ejemplo, si la frecuencia es "muy frecuente", vendrá
    #con un 4 en la celda de relación entre el síntoma y la enfermedad.
    
    
    
    #Ahora calculamos la matriz la similitud entre enfermedades, utilizando la función cosine_similarity (distancia
    #del coseno (vectores))
   
    sim_matrix= sklearn.metrics.pairwise.cosine_similarity(ratings_matrix)
    #Ya tenemos la matriz de similitudes. 
    
    #Ahora ya podemos calcular las recomendaciones. Podemos predecir las enfermedades recomendadas a partir 
    #de una matriz calculada en donde hay relación entre cada enfermedad y síntoma (con un scoring)
    #Es la matriz "sintomas_k"
    #Por eso usamos "ratings_matrix" que tiene las "puntuaciones" entre síntoma y enfermedad
    #Según apuntes... 
    #Cada rating se multiplica por el factor de similitud de enfermedad que dio el rating. 
    #La predicción final por enfermedad será igual a la suma del peso de los ratings dividido por la “suma ponderada”.
    sintomas_k = sim_matrix.dot(ratings_matrix) / np.array([np.abs(sim_matrix).sum(axis=1)]).T
    # Producto de Matriz de similitud (sim_matrix) con la matrix inicial (ratings_matrix)
    #/ (sumatoria de cada fila de ratings) con Transpuesta
    
    print ("matriz de similitudes entre enfermedades. Por eso es 4173x4173")
    print (sim_matrix)
    print (sim_matrix.shape)
    print ("matriz de recomendaciones entre síntoma y enfermedades. Por eso es 4173x483")
    print (sintomas_k)
    print (sintomas_k.shape)
    #Preparamos BBDD
    conn = sqlite3.connect('test_database')
    c = conn.cursor()
    #Hacemos transpose para guardar o dará error en "demasiadas columnas"
    sintomas_k=sintomas_k.transpose()
    df = pd.DataFrame(sintomas_k)
    df=df.transpose()
    df.to_sql('scorings_tfm', conn, if_exists='replace', index = False)
 
   


In [ ]:
init_collaborative_filtering_user_based (df_train_generated.values)


In [ ]:
def predict_collaborative_filtering_user_based(sintoma, elementos):
 
     #Nos viene el Síntoma en la variable "sintoma" (ejemplo "Cough") y el número de elementos enfermedades
    #que queremos que nos recomiende con ese síntoma. Vamos a pensar que son 10 para el ejemplo.
    
    #Sacamos el Id del Sintoma (al tener el texto)
    id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]
    
    
    
    conn = sqlite3.connect('test_database')
    c = conn.cursor()
    c.execute('''  SELECT * FROM scorings_new
          ''')
    #Predicciones (las recomendaciones!)
    #Obtenemos c.fetchall la matriz y montamos dataframe
    sintomas_k = pd.DataFrame(c.fetchall())  
    #transpose porque hemos tenido que hacer transpose para guardar
    sintomas_k=sintomas_k.transpose()
    #ahora la convertimos en array numpy para operar (hacer argsort)
    sintomas_k=sintomas_k.to_numpy()
       
    
    
    #Predicciones (las recomendaciones!)
    
    #Con la matrix de recomendaciones (síntomas vs enfermedades) cogemos únicamente el vector de enfermedades
    #relacionadas con el síntoma "id_sintoma"
    #Pero ojo,    hemos cogido un vector de enfermedades ordenadas!!! 
    sintoma_scoring=sintomas_k.argsort()[id_sintoma]
    #
    vector_id_enfermedad_scoring=[]
    #Inicializamos nuestro vector TOTAL que tendrá las parejas de "enfermedad" /scoring de recomendación
    #(valor en la matriz recomendaciones)
    for i, enfermedad_id in enumerate(sintoma_scoring[-elementos:]):  #Nos cogemos las 10 primeras enfermedades   
        #
        v=[]
    #vamos cogiendo cada pareja de id enfermedad y el scoring de recomendación  (la pareja la montamos en un vector=)
        v.append (enfermedad_id)
        v.append (sintomas_k[id_sintoma][enfermedad_id])
        print ("ranking de peor a mejor")
        print ("posición: ",i)
        print (v)
       
        vector_id_enfermedad_scoring.append(v)  #metemos la pareja en el vector TOTAL
      
    vector_id_enfermedad_scoring=pd.DataFrame(vector_id_enfermedad_scoring)
    #pasamos a data frame y luego montamos con la función interna "aparece y como" una visualización mejor.
    vector_id_enfermedad_scoring = vector_id_enfermedad_scoring.rename(columns={1:id_sintoma, 0:"index"})
    
    listado_completo=_aparece_y_como (vector_id_enfermedad_scoring,id_sintoma, df_Enfermedades)
  

    return listado_completo,sintomas_k, vector_id_enfermedad_scoring

In [ ]:
listado_completo,users_predictions,v=predict_collaborative_filtering_user_based("Cough",10)